# Libraries

In [2]:
# General
import pandas as pd
import numpy as np

# Data Management
from dataclasses import dataclass, field
from collections import namedtuple
import pickle

# Import

Declare Data Structures to hold our data

In [3]:
coordinates = namedtuple("coordinates", ["lat", "lon"])

@dataclass
class class_data:
    name: str
    fluxsat: pd.DataFrame
    climate: pd.DataFrame
    stats: dict
    coords: coordinates

@dataclass
class aes:
    units: dict
    longnames: dict
    abbreviations: dict
    colours: dict

Populate data structures

In [ ]:
# Populate site-specific data structure
df_loc = {
    "se_sto": class_data(
        name = "Abisko-Stordalen Palsa Bog",
        fluxsat = pd.DataFrame(),
        climate = pd.DataFrame(),
        stats = {},
        coords = coordinates(68.35594288, 19.04520892),
    ),
    "fi_ken": class_data(
        name = "Kenttärova",
        fluxsat = pd.DataFrame(),
        climate = pd.DataFrame(),
        stats = {},
        coords = coordinates(67.98721, 24.24301),
    ),
    "fi_sod": class_data(
        name = "Sodankylä",
        fluxsat = pd.DataFrame(),
        climate = pd.DataFrame(),
        stats = {},
        coords = coordinates(67.36239, 26.63859),
    ),
    "fi_var": class_data(
        name = "Varrio",
        fluxsat = pd.DataFrame(),
        climate = pd.DataFrame(),
        stats = {},
        coords = coordinates(67.7549, 29.61),
    )
}

# Read in site-specific datatables
for key, item in df_loc.items():
    item.fluxsat = pd.read_csv(f"../data/processed/{key}_fluxsat.csv", index_col = 0)
    item.climate = pd.read_csv(f"../data/processed/{key}_climate.csv", index_col = 0)

In [16]:
df_loc["se_sto"].fluxsat

,datetime,day_of_year,time_of_day,gppe,swce
1981-07-26 03:20:51.529411682,1981-07-26 03:20:51.529411682,207,2025-05-15 03:20:51.529411,2.417515,9.834162
1981-08-21 12:56:24.000000000,1981-08-21 12:56:24.000000000,233,2025-05-15 12:56:24.000000,2.758144,9.755122
1981-09-04 13:39:36.000000000,1981-09-04 13:39:36.000000000,247,2025-05-15 13:39:36.000000,1.546780,10.213472
1981-09-08 12:53:24.000000000,1981-09-08 12:53:24.000000000,251,2025-05-15 12:53:24.000000,1.820904,10.253892
1981-09-13 13:37:48.000000000,1981-09-13 13:37:48.000000000,256,2025-05-15 13:37:48.000000,1.691738,9.080672
...,...,...,...,...,...
2025-05-04 10:52:09.067000000,2025-05-04 10:52:09.067000000,124,2025-05-15 10:52:09.067000,0.155096,31.320820
2025-05-06 10:41:49.113000000,2025-05-06 10:41:49.113000000,126,2025-05-15 10:41:49.113000,0.116298,38.090785
2025-05-06 10:41:52.945000000,2025-05-06 10:41:52.945000000,126,2025-05-15 10:41:52.945000,0.110607,37.900843
2025-05-06 10:51:56.778000000,2025-05-06 10:51:56.778000000,126,2025-05-15 10:51:56.778000,0.141514,33.525374


# Normality

# Statistics

1. GPP correlate with SWC
2. GPP correlate with climate variables
3. NOA/AO correlate with climate variables
4. NOA/AO correlate with GPP

# Figures

Save to: "../figures/"